# Gram–Schmidt orthogonalization

Chapter 4.4 illustrates a hand technique for computing orthonormal vectors q₁,q₂,… from arbitrary vectors a,b,… with the property that the first k vectors in the original set span the same subspace as the orthonormal set, and this is true for k=1,2,3,...

We will move this hand technique to the computer in this notebook.  Some of you will notice that on the computer one can combine operations in a simpler block fashion.  

In [1]:
# start with four arbitrary independent vectors in ℝᵐ
# with random entries from 1 to 10.
m = 6
a₁ = rand(1:10,m)
a₂ = rand(1:10,m)
a₃ = rand(1:10,m)
a₄ = rand(1:10,m)
A = [a₁ a₂ a₃ a₄] # show them as the columns of a 6×4 matrix A

6×4 Array{Int64,2}:
 7  2  6   9
 2  4  3   2
 8  6  4  10
 4  9  2   5
 7  1  5   9
 7  8  4   6

In [2]:
# The vₖ are vectors, but they are all orthogonal and
#span([v₁]) = span([a₁])
#span([v₁ v₂]) = span([a₁ a₂])
#span([v₁ v₂ v₃]) = span([a₁ a₂ a₃] )
#span([v₁ v₂ v₃ v₄]) = span([a₁ a₂ a₃ a₄])
v₁ = a₁
v₂ = a₂ - v₁*(v₁'a₂)/(v₁'v₁)
v₃ = a₃ - v₁*(v₁'a₃)/(v₁'v₁) - v₂*(v₂'a₃)/(v₂'v₂)
v₄ = a₄ - v₁*(v₁'a₄)/(v₁'v₁) - v₂*(v₂'a₄)/(v₂'v₂) - v₃*(v₃'a₄)/(v₃'v₃)

# gather into a matrix V with orthogonal but *not* orthonormal columns
V = [v₁ v₂ v₃ v₄]

6×4 Array{Float64,2}:
 7.0  -3.12121    1.12662     0.293808 
 2.0   2.5368     1.93453    -0.0432856
 8.0   0.147186  -1.2154      0.62856  
 4.0   6.07359   -0.0355781   1.14313  
 7.0  -4.12121    0.031269    0.178556 
 7.0   2.87879   -0.301254   -1.83157  

In [3]:
# now we normalize
q₁ = normalize(v₁) # same as A/norm(A)
q₂ = normalize(v₂)
q₃ = normalize(v₃)
q₄ = normalize(v₄);

# Gather into a matrix Q with orthonormal columns
Q = [q₁ q₂ q₃ q₄]

6×4 Array{Float64,2}:
 0.460566  -0.352596    0.43914     0.129135 
 0.13159    0.286576    0.754051   -0.0190249
 0.526361   0.0166273  -0.473745    0.276266 
 0.263181   0.686121   -0.0138678   0.502429 
 0.460566  -0.465564    0.0121882   0.0784793
 0.460566   0.32521    -0.117424   -0.805013 

In [4]:
#check that Q has orthonormal columns
Q'Q

4×4 Array{Float64,2}:
  1.0           4.06e-17     -9.50947e-17  -7.11172e-17
  4.06e-17      1.0           7.34308e-17  -1.81051e-16
 -9.50947e-17   7.34308e-17   1.0           1.19386e-15
 -7.11172e-17  -1.81051e-16   1.19386e-15   1.0        

In [5]:
Q'Q - I

4×4 Array{Float64,2}:
  0.0           4.06e-17     -9.50947e-17  -7.11172e-17
  4.06e-17      2.22045e-16   7.34308e-17  -1.81051e-16
 -9.50947e-17   7.34308e-17   0.0           1.19386e-15
 -7.11172e-17  -1.81051e-16   1.19386e-15   2.22045e-16

In [6]:
Q'Q ≈ eye(4)

true

In [7]:
# compare to what happens if we didn't normalize:
V'V # = diagonal matrix (orthogonal columns, but not orthonormal)

4×4 Array{Float64,2}:
 231.0           3.55271e-15  -4.88498e-15  -3.10862e-15
   3.55271e-15  78.3593        1.26619e-15  -2.82135e-15
  -4.88498e-15   1.26619e-15   6.5819        6.95173e-15
  -3.10862e-15  -2.82135e-15   6.95173e-15   5.17654    

In [8]:
# What does this triangular structure say?
round.(Q'A, 5)

4×4 Array{Float64,2}:
 15.1987  11.1194    9.93507  17.8963 
  0.0      8.85208  -0.84408  -1.24216
 -0.0     -0.0       2.56552   0.05872
 -0.0     -0.0       0.0       2.2752 

## QR factorization

How do we do all this at once on a computer? We ask the computer to factor the matrix as $QR$ (orthonormal columns times upper triangular).

In [9]:
Q2,R = qr(A)

([-0.460566 0.352596 0.43914 -0.129135; -0.13159 -0.286576 0.754051 0.0190249; … ; -0.460566 0.465564 0.0121882 -0.0784793; -0.460566 -0.32521 -0.117424 0.805013], [-15.1987 -11.1194 -9.93507 -17.8963; 0.0 -8.85208 0.84408 1.24216; 0.0 0.0 2.56552 0.0587228; 0.0 0.0 0.0 -2.2752])

In [10]:
round.(Q'Q2, 5) # almost I, up to signs

4×4 Array{Float64,2}:
 -1.0   0.0   0.0   0.0
 -0.0  -1.0  -0.0   0.0
  0.0  -0.0   1.0  -0.0
  0.0   0.0   0.0  -1.0

In [11]:
R # Recognize this matrix?

4×4 Array{Float64,2}:
 -15.1987  -11.1194   -9.93507  -17.8963   
   0.0      -8.85208   0.84408    1.24216  
   0.0       0.0       2.56552    0.0587228
   0.0       0.0       0.0       -2.2752   

In [12]:
Q2*R ≈ A

true

In [14]:
b = rand(6)

6-element Array{Float64,1}:
 0.891932
 0.169511
 0.408775
 0.138505
 0.768323
 0.147395

In [15]:
A \ b

4-element Array{Float64,1}:
 -0.108273 
 -0.0274501
  0.12079  
  0.103778 

In [16]:
inv(A'A) * A'b

4-element Array{Float64,1}:
 -0.108273 
 -0.0274501
  0.12079  
  0.103778 

In [17]:
R \ Q2'b

4-element Array{Float64,1}:
 -0.108273 
 -0.0274501
  0.12079  
  0.103778 